In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, mean_squared_error

In [2]:
class MovieLensDataset(torch.utils.data.Dataset):
    def __init__(self, ratings):
        super().__init__()
        data = ratings.copy().to_numpy()
        self.items = data[:, :2].astype(np.int32) - 1
        self.targets = self.__preprocess_targets(data[:, 2]).astype(np.float32)
        self.field_dims = np.max(self.items, axis=0) + 1
        self.user_field_idx = np.array((0,), dtype=np.int64)
        self.item_field_idx = np.array((1,), dtype=np.int64)

    def __len__(self):
        return self.targets.shape[0]

    def __getitem__(self, idx):
        return self.items[idx], self.targets[idx]
    
    def __preprocess_targets(self, target):
        target = target / 5.
        return target


In [3]:
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
        super().__init__()
        layers = list()
        for embed_dim in embed_dims:
            layers.append(torch.nn.Linear(input_dim, embed_dim))
            layers.append(torch.nn.BatchNorm1d(embed_dim))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=dropout))
            input_dim = embed_dim
        if output_layer:
            layers.append(torch.nn.Linear(input_dim, 1))
        self.mlp = torch.nn.Sequential(*layers)
    
    def forward(self, x):
        return self.mlp(x)

In [4]:
class FeatureEmbedding(torch.nn.Module):
    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x):
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)

In [5]:
class NeuralCollaborativeFiltering(torch.nn.Module):
    def __init__(self, field_dims, user_field_idx, item_field_idx, embed_dim, mlp_dims, dropout):
        super().__init__()
        self.user_field_idx = user_field_idx
        self.item_field_idx = item_field_idx
        self.embedding = FeatureEmbedding(field_dims, embed_dim)
        self.embed_output_dim = len(field_dims) * embed_dim
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout, output_layer=False)
        self.fc = torch.nn.Linear(mlp_dims[-1] + embed_dim, 1)

    def forward(self, x):
        x = self.embedding(x)
        user_x = x[:, self.user_field_idx].squeeze(1)
        item_x = x[:, self.item_field_idx].squeeze(1)
        x = self.mlp(x.view(-1, self.embed_output_dim))
        gmf = user_x * item_x
        x = torch.cat([gmf, x], dim=1)
        x = self.fc(x).squeeze(1)
        return torch.sigmoid(x)

In [6]:
def train(model, optimizer, data_loader, criterion, device, log_interval=100):
    model.train()
    total_loss = 0
    av_loss = []
    for i, (fields, target) in enumerate(data_loader):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    loss = total_loss / log_interval
    av_loss.append(loss)
    total_loss = 0
    return np.mean(av_loss)

In [7]:
def test(model, data_loader, device):
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for fields, target in data_loader:
            fields, target = fields.to(device), target.to(device)
            y = model(fields)
            targets.extend(target.tolist())
            predicts.extend(y.tolist())
    return 5. * mean_squared_error(targets, predicts)

In [8]:
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
dataset = MovieLensDataset(ratings)

In [9]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
learning_rate = 0.001
weight_decay = 1e-6
batch_size = 2048
epochs = 10
model_name = "ncf"


In [10]:
train_length = int(len(dataset) * 0.8)
valid_length = int(len(dataset) * 0.1)
test_length = len(dataset) - train_length - valid_length

train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset,
                                                                           (train_length, valid_length, test_length))

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=2)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=2)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2)


In [11]:
model = NeuralCollaborativeFiltering(
    dataset.field_dims,
    embed_dim=64,
    mlp_dims=(32, 32),
    dropout=0.2,
    user_field_idx=dataset.user_field_idx,
    item_field_idx=dataset.item_field_idx
).to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [12]:
# metric_values = []
# loss_values = []
# 
# for epoch_i in tqdm(range(epochs)):
#     loss = train(model, optimizer, train_data_loader, criterion, device)
#     loss_values.append((epoch_i, loss))
#     metric_valid = test(model, valid_data_loader, device)
#  
#     metric_train = test(model, train_data_loader, device)
#     metric_test = test(model, test_data_loader, device)
# 
#     metric_values.append((epoch_i, metric_train, metric_valid, metric_test))

In [14]:
list(dataset.field_dims)

[671, 163949]

In [15]:
metric_values: list[float] =[]
loss_values: list[float] = []
model_save_path = "../data/model.pth"
best_valid_loss = float('inf')

for epoch_i in range(epochs):
    loss = train(model, optimizer, train_data_loader, criterion, device)
    loss_values.append((epoch_i, loss))
    
    valid_loss = test(model, valid_data_loader, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved at epoch {epoch_i}")
        
    metric_valid = test(model, valid_data_loader, device)
    metric_train = test(model, train_data_loader, device)
    metric_test = test(model, test_data_loader, device)
     
    metric_values.append((epoch_i, metric_train, metric_valid, metric_test))

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/jaypark/anaconda3/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jaypark/anaconda3/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'MovieLensDataset' on <module '__main__' (built-in)>


KeyboardInterrupt: 